In [1023]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from skimpy import clean_columns, skim
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.decomposition import PCA


Downloading the data    

In [1024]:

brent = clean_columns (pd.read_csv("new-shell-cashflow-datathon-2023/brent.csv", parse_dates= ['Tarih']))
cash_flow_train = clean_columns(pd.read_csv("new-shell-cashflow-datathon-2023/cash_flow_train.csv", parse_dates= ['Date']))  
depo_pump_imm =  clean_columns(pd.read_csv("new-shell-cashflow-datathon-2023/depo_pump_imm.csv")) 
otv =  clean_columns(pd.read_csv("new-shell-cashflow-datathon-2023/otv.csv", parse_dates= ['Tarih'])) 
platts = clean_columns(pd.read_csv("new-shell-cashflow-datathon-2023/platts.csv", parse_dates= ['Tarih']))  
sample_submission = pd.read_csv("new-shell-cashflow-datathon-2023/sample_submission.csv")
usd =  clean_columns(pd.read_csv("new-shell-cashflow-datathon-2023/usd.csv", parse_dates= ['Tarih'])) 
volume = clean_columns(pd.read_csv("new-shell-cashflow-datathon-2023/volume.csv", parse_dates= ['Posting date']))

7 column names have been cleaned

17 column names have been cleaned

5 column names have been cleaned

5 column names have been cleaned

7 column names have been cleaned

8 column names have been cleaned

5 column names have been cleaned

In [1025]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 2 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Date                          70 non-null     object
 1   Net Cashflow from Operations  70 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.2+ KB


In [1026]:
cash_flow_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            1025 non-null   datetime64[ns]
 1   total_inflows                   1025 non-null   float64       
 2   customers_dds                   1025 non-null   float64       
 3   customers_eft                   1025 non-null   float64       
 4   t_&_s_collections               1025 non-null   float64       
 5   fx_sales                        1025 non-null   float64       
 6   other_operations                1025 non-null   float64       
 7   total_outflows                  1025 non-null   float64       
 8   tupras                          1025 non-null   float64       
 9   other_oil                       1025 non-null   float64       
 10  gas                             1025 non-null   float64       
 11  impo

In [1027]:
cash_flow_train.describe

<bound method NDFrame.describe of            date  total_inflows  customers_dds  customers_eft  \
0    2019-01-02   1.747050e+07      7718750.0   9.655500e+06   
1    2019-01-03   1.153048e+07      3431500.0   8.098978e+06   
2    2019-01-04   2.591328e+07     13775750.0   1.213753e+07   
3    2019-01-07   5.259345e+07     39041750.0   1.355170e+07   
4    2019-01-08   3.650429e+07     15137500.0   2.136679e+07   
...         ...            ...            ...            ...   
1020 2023-01-26   1.408741e+08     90239000.0   4.977288e+07   
1021 2023-01-27   1.062562e+08     53296500.0   5.282375e+07   
1022 2023-01-30   5.225381e+08    350467750.0   1.679393e+08   
1023 2023-01-31   2.323393e+08    131395500.0   9.922482e+07   
1024 2023-02-01   9.117482e+07     56928500.0   3.354297e+07   

      t_&_s_collections  fx_sales  other_operations  total_outflows  \
0          0.000000e+00       0.0           96250.0   -4.631593e+07   
1          0.000000e+00       0.0               0.0   -

In [1028]:
brent.describe

<bound method NDFrame.describe of           tarih       urun avrupa_birligi_para_birimi  ab_piyasa_fiyat  \
0    2023-02-01  ICE BRENT                      $/BBL            82.84   
1    2023-01-31  ICE BRENT                      $/BBL            84.49   
2    2023-01-30  ICE BRENT                      $/BBL            84.90   
3    2023-01-29  ICE BRENT                      $/BBL            86.66   
4    2023-01-28  ICE BRENT                      $/BBL            86.66   
...         ...        ...                        ...              ...   
1852 2018-01-06  ICE BRENT                      $/BBL            67.62   
1853 2018-01-05  ICE BRENT                      $/BBL            67.62   
1854 2018-01-04  ICE BRENT                      $/BBL            68.07   
1855 2018-01-03  ICE BRENT                      $/BBL            67.84   
1856 2018-01-02  ICE BRENT                      $/BBL            66.57   

      ab_piyasa_fiyat_yuksek  ab_piyasa_fiyat_dusuk  dolar_kuru_sats  
0     

In [1029]:
brent_cash = cash_flow_train.merge(brent, left_on= 'date', right_on='tarih', how= 'left').drop('tarih', axis =1)


In [1030]:
brent_cash

,date,total_inflows,customers_dds,customers_eft,t_&_s_collections,fx_sales,other_operations,total_outflows,tupras,other_oil,...,operational_and_admin_expenses,vis_buyback_payments,net_cashflow_from_operations,inflows_currency,urun,avrupa_birligi_para_birimi,ab_piyasa_fiyat,ab_piyasa_fiyat_yuksek,ab_piyasa_fiyat_dusuk,dolar_kuru_sats
0,2019-01-02,1.747050e+07,7718750.0,9.655500e+06,0.000000e+00,0.0,96250.0,-4.631593e+07,-1.457022e+07,-1.441510e+05,...,-1.112469e+06,0.000000e+00,-2.884543e+07,NaN,ICE BRENT,$/BBL,54.91,56.56,52.51,5.3412
1,2019-01-03,1.153048e+07,3431500.0,8.098978e+06,0.000000e+00,0.0,0.0,-1.535349e+07,-1.337181e+07,-3.007992e+05,...,-6.044857e+05,0.000000e+00,-3.823007e+06,NaN,ICE BRENT,$/BBL,55.95,56.30,53.93,5.4686
2,2019-01-04,2.591328e+07,13775750.0,1.213753e+07,0.000000e+00,0.0,0.0,-1.098651e+08,-2.459390e+07,0.000000e+00,...,-7.175890e+06,-2.612621e+07,-8.395180e+07,NaN,ICE BRENT,$/BBL,57.06,58.31,55.34,5.4186
3,2019-01-07,5.259345e+07,39041750.0,1.355170e+07,0.000000e+00,0.0,0.0,-5.165489e+07,-3.110594e+07,-3.364373e+06,...,-2.906375e+06,0.000000e+00,9.385578e+05,NaN,ICE BRENT,$/BBL,57.33,58.93,57.26,5.3693
4,2019-01-08,3.650429e+07,15137500.0,2.136679e+07,0.000000e+00,0.0,0.0,-1.842178e+07,-1.246099e+07,-1.676525e+04,...,-1.030241e+06,0.000000e+00,1.808252e+07,NaN,ICE BRENT,$/BBL,58.72,58.87,57.13,5.4362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,2023-01-26,1.408741e+08,90239000.0,4.977288e+07,8.622500e+05,0.0,0.0,-1.318398e+08,-4.748971e+07,-3.760017e+05,...,-2.771415e+06,-1.500000e+06,9.034332e+06,12500.0,ICE BRENT,$/BBL,87.47,88.01,85.74,18.8191
1021,2023-01-27,1.062562e+08,53296500.0,5.282375e+07,1.359018e+05,0.0,0.0,-2.777992e+08,-4.800505e+07,-1.360219e+08,...,-9.688081e+06,-2.030862e+06,-1.715431e+08,0.0,ICE BRENT,$/BBL,86.66,88.95,85.69,18.8191
1022,2023-01-30,5.225381e+08,350467750.0,1.679393e+08,4.131052e+06,0.0,0.0,-3.223933e+08,-1.854239e+08,-2.196258e+07,...,-1.519839e+07,0.000000e+00,2.001448e+08,0.0,ICE BRENT,$/BBL,84.90,87.48,84.66,18.8215
1023,2023-01-31,2.323393e+08,131395500.0,9.922482e+07,1.718965e+06,0.0,0.0,-1.187000e+08,-3.198080e+07,-1.251479e+06,...,-7.624322e+06,0.000000e+00,1.136393e+08,0.0,ICE BRENT,$/BBL,84.49,85.25,83.73,18.8216


In [1031]:
dates = pd.DataFrame({'date' :pd.date_range(start='1-1-2018', end= '31-12-2022')})  
dates['year'] = dates['date'].dt.year
dates['month'] = dates['date'].dt.month

/var/folders/t1/db5m68vn10v3148_nq_7r8b80000gn/T/ipykernel_1061/870535568.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  dates = pd.DataFrame({'date' :pd.date_range(start='1-1-2018', end= '31-12-2022')})


In [1032]:
depo = dates.merge(depo_pump_imm, left_on= ['year', 'month'], right_on= ['yl', 'ay']).drop(['yl','ay'], axis = 1)

In [1033]:
brent_cash_depo = brent_cash.merge(depo, on = 'date', how = 'left')

In [1034]:
brent_cash_depo 

,date,total_inflows,customers_dds,customers_eft,t_&_s_collections,fx_sales,other_operations,total_outflows,tupras,other_oil,...,avrupa_birligi_para_birimi,ab_piyasa_fiyat,ab_piyasa_fiyat_yuksek,ab_piyasa_fiyat_dusuk,dolar_kuru_sats,year,month,imm_tl_m_3,pump_price,depot_price
0,2019-01-02,1.747050e+07,7718750.0,9.655500e+06,0.000000e+00,0.0,96250.0,-4.631593e+07,-1.457022e+07,-1.441510e+05,...,$/BBL,54.91,56.56,52.51,5.3412,2019.0,1.0,"723,05","6,02","4.784,03"
1,2019-01-03,1.153048e+07,3431500.0,8.098978e+06,0.000000e+00,0.0,0.0,-1.535349e+07,-1.337181e+07,-3.007992e+05,...,$/BBL,55.95,56.30,53.93,5.4686,2019.0,1.0,"723,05","6,02","4.784,03"
2,2019-01-04,2.591328e+07,13775750.0,1.213753e+07,0.000000e+00,0.0,0.0,-1.098651e+08,-2.459390e+07,0.000000e+00,...,$/BBL,57.06,58.31,55.34,5.4186,2019.0,1.0,"723,05","6,02","4.784,03"
3,2019-01-07,5.259345e+07,39041750.0,1.355170e+07,0.000000e+00,0.0,0.0,-5.165489e+07,-3.110594e+07,-3.364373e+06,...,$/BBL,57.33,58.93,57.26,5.3693,2019.0,1.0,"723,05","6,02","4.784,03"
4,2019-01-08,3.650429e+07,15137500.0,2.136679e+07,0.000000e+00,0.0,0.0,-1.842178e+07,-1.246099e+07,-1.676525e+04,...,$/BBL,58.72,58.87,57.13,5.4362,2019.0,1.0,"723,05","6,02","4.784,03"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,2023-01-26,1.408741e+08,90239000.0,4.977288e+07,8.622500e+05,0.0,0.0,-1.318398e+08,-4.748971e+07,-3.760017e+05,...,$/BBL,87.47,88.01,85.74,18.8191,NaN,NaN,NaN,NaN,NaN
1021,2023-01-27,1.062562e+08,53296500.0,5.282375e+07,1.359018e+05,0.0,0.0,-2.777992e+08,-4.800505e+07,-1.360219e+08,...,$/BBL,86.66,88.95,85.69,18.8191,NaN,NaN,NaN,NaN,NaN
1022,2023-01-30,5.225381e+08,350467750.0,1.679393e+08,4.131052e+06,0.0,0.0,-3.223933e+08,-1.854239e+08,-2.196258e+07,...,$/BBL,84.90,87.48,84.66,18.8215,NaN,NaN,NaN,NaN,NaN
1023,2023-01-31,2.323393e+08,131395500.0,9.922482e+07,1.718965e+06,0.0,0.0,-1.187000e+08,-3.198080e+07,-1.251479e+06,...,$/BBL,84.49,85.25,83.73,18.8216,NaN,NaN,NaN,NaN,NaN


In [1035]:
otv.head()

,tarih,otv,adet,otv_oran,otv_uygulanan_urun_ad
0,2018-05-22,2124.10,1000,2.12410,Kursunsuz Benzin 95
1,2018-05-22,2124.10,1000,2.12410,Kursunsuz Benzin 95
2,2018-05-22,2081.62,1000,2.08162,Etanollü Kurşunsuz Benzin 95
3,2018-05-22,2081.62,1000,2.08162,Etanollü Kurşunsuz Benzin 95
4,2018-05-22,1567.90,1000,1.56790,Motorin


In [1036]:
otv.drop_duplicates(inplace= True)

In [1037]:
otv_fixed =  otv.drop(['otv','adet'], axis = 1).pivot(index= 'tarih', columns= 'otv_uygulanan_urun_ad').reset_index()

In [1038]:
otv_fixed = otv_fixed.droplevel(0, axis =1)

In [1039]:
otv_fixed.columns = ['date', 'benzinli_motorin', 'etanallu_kursunsuz_benzin_95', 'kursunsuz_benzin_95','lpg','motorin']

In [1040]:
otv_fixed.fillna(method= 'bfill').fillna(method= 'ffill', inplace= True )

In [1041]:
brent_cash_depo_otv =  brent_cash_depo.merge(otv_fixed,left_on = 'date', right_on = 'date', how = 'left')

In [1042]:
brent_cash_depo_otv.head()

,date,total_inflows,customers_dds,customers_eft,t_&_s_collections,fx_sales,other_operations,total_outflows,tupras,other_oil,...,year,month,imm_tl_m_3,pump_price,depot_price,benzinli_motorin,etanallu_kursunsuz_benzin_95,kursunsuz_benzin_95,lpg,motorin
0,2019-01-02,1.747050e+07,7718750.0,9.655500e+06,0.0,0.0,96250.0,-4.631593e+07,-1.457022e+07,-144151.00,...,2019.0,1.0,"723,05","6,02","4.784,03",NaN,NaN,NaN,NaN,NaN
1,2019-01-03,1.153048e+07,3431500.0,8.098978e+06,0.0,0.0,0.0,-1.535349e+07,-1.337181e+07,-300799.25,...,2019.0,1.0,"723,05","6,02","4.784,03",NaN,NaN,NaN,NaN,NaN
2,2019-01-04,2.591328e+07,13775750.0,1.213753e+07,0.0,0.0,0.0,-1.098651e+08,-2.459390e+07,0.00,...,2019.0,1.0,"723,05","6,02","4.784,03",NaN,NaN,NaN,NaN,NaN
3,2019-01-07,5.259345e+07,39041750.0,1.355170e+07,0.0,0.0,0.0,-5.165489e+07,-3.110594e+07,-3364373.00,...,2019.0,1.0,"723,05","6,02","4.784,03",NaN,NaN,NaN,NaN,NaN
4,2019-01-08,3.650429e+07,15137500.0,2.136679e+07,0.0,0.0,0.0,-1.842178e+07,-1.246099e+07,-16765.25,...,2019.0,1.0,"723,05","6,02","4.784,03",NaN,NaN,NaN,NaN,NaN


In [1043]:
platts_1 =  platts[platts['urun'] == '10 ppm ULSD CIF Med (Genova/Lavera)']

In [1044]:
platts_2 = platts[platts['urun'] == 'Prem Unl 10 ppm CIF Med (Genova/Lavera)']

In [1045]:
brent_cash_depo_otv_platts_1 =  brent_cash_depo_otv.merge(platts_1,left_on = 'date', right_on = 'tarih', how = 'left')

In [1046]:
brent_cash_depo_otv_platts =  brent_cash_depo_otv_platts_1.merge(platts_2,left_on = 'date', right_on = 'tarih', how = 'left')

In [1047]:
brent_cash_depo_otv_platts

,date,total_inflows,customers_dds,customers_eft,t_&_s_collections,fx_sales,other_operations,total_outflows,tupras,other_oil,...,ab_piyasa_fiyat_yuksek_y,ab_piyasa_fiyat_dusuk_y,dolar_kuru_sats_y,tarih_y,urun,avrupa_birligi_birimi_y,ab_piyasa_fiyat,ab_piyasa_fiyat_yuksek,ab_piyasa_fiyat_dusuk,dolar_kuru_sats
0,2019-01-02,1.747050e+07,7718750.0,9.655500e+06,0.000000e+00,0.0,96250.0,-4.631593e+07,-1.457022e+07,-1.441510e+05,...,521.50,521.00,5.3412,2019-01-02,Prem Unl 10 ppm CIF Med (Genova/Lavera),$/TON,485.50,485.75,485.25,5.3412
1,2019-01-03,1.153048e+07,3431500.0,8.098978e+06,0.000000e+00,0.0,0.0,-1.535349e+07,-1.337181e+07,-3.007992e+05,...,517.75,517.25,5.4686,2019-01-03,Prem Unl 10 ppm CIF Med (Genova/Lavera),$/TON,475.25,475.50,475.00,5.4686
2,2019-01-04,2.591328e+07,13775750.0,1.213753e+07,0.000000e+00,0.0,0.0,-1.098651e+08,-2.459390e+07,0.000000e+00,...,534.75,534.25,5.4186,2019-01-04,Prem Unl 10 ppm CIF Med (Genova/Lavera),$/TON,488.00,488.25,487.75,5.4186
3,2019-01-07,5.259345e+07,39041750.0,1.355170e+07,0.000000e+00,0.0,0.0,-5.165489e+07,-3.110594e+07,-3.364373e+06,...,550.00,549.50,5.3693,2019-01-07,Prem Unl 10 ppm CIF Med (Genova/Lavera),$/TON,496.50,496.75,496.25,5.3693
4,2019-01-08,3.650429e+07,15137500.0,2.136679e+07,0.000000e+00,0.0,0.0,-1.842178e+07,-1.246099e+07,-1.676525e+04,...,551.50,551.00,5.4362,2019-01-08,Prem Unl 10 ppm CIF Med (Genova/Lavera),$/TON,491.75,492.00,491.50,5.4362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,2023-01-26,1.408741e+08,90239000.0,4.977288e+07,8.622500e+05,0.0,0.0,-1.318398e+08,-4.748971e+07,-3.760017e+05,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1021,2023-01-27,1.062562e+08,53296500.0,5.282375e+07,1.359018e+05,0.0,0.0,-2.777992e+08,-4.800505e+07,-1.360219e+08,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1022,2023-01-30,5.225381e+08,350467750.0,1.679393e+08,4.131052e+06,0.0,0.0,-3.223933e+08,-1.854239e+08,-2.196258e+07,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1023,2023-01-31,2.323393e+08,131395500.0,9.922482e+07,1.718965e+06,0.0,0.0,-1.187000e+08,-3.198080e+07,-1.251479e+06,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [1048]:
brent_cash_depo_otv_platts

,date,total_inflows,customers_dds,customers_eft,t_&_s_collections,fx_sales,other_operations,total_outflows,tupras,other_oil,...,ab_piyasa_fiyat_yuksek_y,ab_piyasa_fiyat_dusuk_y,dolar_kuru_sats_y,tarih_y,urun,avrupa_birligi_birimi_y,ab_piyasa_fiyat,ab_piyasa_fiyat_yuksek,ab_piyasa_fiyat_dusuk,dolar_kuru_sats
0,2019-01-02,1.747050e+07,7718750.0,9.655500e+06,0.000000e+00,0.0,96250.0,-4.631593e+07,-1.457022e+07,-1.441510e+05,...,521.50,521.00,5.3412,2019-01-02,Prem Unl 10 ppm CIF Med (Genova/Lavera),$/TON,485.50,485.75,485.25,5.3412
1,2019-01-03,1.153048e+07,3431500.0,8.098978e+06,0.000000e+00,0.0,0.0,-1.535349e+07,-1.337181e+07,-3.007992e+05,...,517.75,517.25,5.4686,2019-01-03,Prem Unl 10 ppm CIF Med (Genova/Lavera),$/TON,475.25,475.50,475.00,5.4686
2,2019-01-04,2.591328e+07,13775750.0,1.213753e+07,0.000000e+00,0.0,0.0,-1.098651e+08,-2.459390e+07,0.000000e+00,...,534.75,534.25,5.4186,2019-01-04,Prem Unl 10 ppm CIF Med (Genova/Lavera),$/TON,488.00,488.25,487.75,5.4186
3,2019-01-07,5.259345e+07,39041750.0,1.355170e+07,0.000000e+00,0.0,0.0,-5.165489e+07,-3.110594e+07,-3.364373e+06,...,550.00,549.50,5.3693,2019-01-07,Prem Unl 10 ppm CIF Med (Genova/Lavera),$/TON,496.50,496.75,496.25,5.3693
4,2019-01-08,3.650429e+07,15137500.0,2.136679e+07,0.000000e+00,0.0,0.0,-1.842178e+07,-1.246099e+07,-1.676525e+04,...,551.50,551.00,5.4362,2019-01-08,Prem Unl 10 ppm CIF Med (Genova/Lavera),$/TON,491.75,492.00,491.50,5.4362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,2023-01-26,1.408741e+08,90239000.0,4.977288e+07,8.622500e+05,0.0,0.0,-1.318398e+08,-4.748971e+07,-3.760017e+05,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1021,2023-01-27,1.062562e+08,53296500.0,5.282375e+07,1.359018e+05,0.0,0.0,-2.777992e+08,-4.800505e+07,-1.360219e+08,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1022,2023-01-30,5.225381e+08,350467750.0,1.679393e+08,4.131052e+06,0.0,0.0,-3.223933e+08,-1.854239e+08,-2.196258e+07,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1023,2023-01-31,2.323393e+08,131395500.0,9.922482e+07,1.718965e+06,0.0,0.0,-1.187000e+08,-3.198080e+07,-1.251479e+06,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [1049]:
brent_cash_depo_otv_platts_usd =  brent_cash_depo_otv_platts.merge(usd,left_on = 'date', right_on = 'tarih', how = 'left')

In [1050]:
skim(brent_cash_depo_otv_platts_usd)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 1025   │ │ float64     │ 41    │                                                          │
│ │ Number of columns │ 55     │ │ string      │ 9     │                                                          │
│ └───────────────────┴────────┘ │ datetime64  │ 4     │                                                          │
│                                │ int64       │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_nam ┃      ┃      ┃           ┃           ┃            ┃           ┃           ┃           ┃        ┃  │
│ ┃ e          ┃ NA   ┃ NA % ┃ mean      ┃ sd        ┃ p0         ┃ p25       ┃ p75       ┃ p100      ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━┩  │
│ │ total_infl │    0 │    0 │ 100000000 │ 110000000 │    8800000 │  38000000 │ 120000000 │ 860000000 │   █▁   │  │
│ │ ows        │      │      │           │           │            │           │           │           │        │  │
│ │ customers_ │    0 │    0 │  48000000 │  59000000 │    2500000 │  15000000 │  58000000 │ 570000000 │   █▁   │  │
│ │ dds        │      │      │           │           │            │           │           │           │        │  │
│ │ customers_ │    0 │    0 │  38000000 │  40000000 │    4600000 │  15000000 │  47000000 │ 330000000 │   █▁   │  │
│ │ eft        │      │      │           │           │            │           │           │           │        │  │
│ │ t_&_s_coll │    0 │    0 │  11000000 │   9600000 │          0 │   5000000 │  14000000 │  79000000 │   █▂   │  │
│ │ ections    │      │      │           │           │            │           │           │           │        │  │
│ │ fx_sales   │    0 │    0 │    410000 │   4900000 │  -11000000 │         0 │         0 │ 140000000 │   █    │  │
│ │ other_oper │    0 │    0 │   2100000 │   8600000 │   -1200000 │         0 │         0 │ 100000000 │   █    │  │
│ │ ations     │      │      │           │           │            │           │           │           │        │  │
│ │ total_outf │    0 │    0 │ -86000000 │  88000000 │ -590000000 │ -11000000 │ -29000000 │  -5800000 │    ▁▂█ │  │
│ │ lows       │      │      │           │           │            │         0 │           │           │        │  │
│ │ tupras     │    0 │    0 │ -34000000 │  40000000 │ -300000000 │ -43000000 │ -11000000 │   -790000 │     ▁█ │  │
│ │ other_oil  │    0 │    0 │ -14000000 │  25000000 │ -210000000 │ -15000000 │  -2600000 │         0 │      █ │  │
│ │ gas        │    0 │    0 │  -4200000 │   5700000 │  -45000000 │  -5300000 │  -1000000 │         0 │     ▁█ │  │
│ │ import_pay │    0 │    0 │ -16000000 │  27000000 │ -320000000 │ -25000000 │         0 │         0 │     ▁█ │  │
│ │ ments_fx_p │      │      │           │           │            │           │           │           │        │  │
│ │ tax        │    0 │    0 │  -8000000 │  16000000 │ -130000000 │  -5900000 │   -180000 │         0 │     ▁█ │  │
│ │ operationa │    0 │    0 │  -4100000 │  10000000 │ -

In [1051]:
brent_cash_depo_otv_platts_usd


,date,total_inflows,customers_dds,customers_eft,t_&_s_collections,fx_sales,other_operations,total_outflows,tupras,other_oil,...,ab_piyasa_fiyat_dusuk,dolar_kuru_sats,yl,tarih,usd_alis,usd_satis,eur_alis,eur_satis,gbp_alis,gbp_satis
0,2019-01-02,1.747050e+07,7718750.0,9.655500e+06,0.000000e+00,0.0,96250.0,-4.631593e+07,-1.457022e+07,-1.441510e+05,...,485.25,5.3412,2019,2019-01-02,5.2810,5.2905,6.0422,6.0530,6.7135,6.7485
1,2019-01-03,1.153048e+07,3431500.0,8.098978e+06,0.000000e+00,0.0,0.0,-1.535349e+07,-1.337181e+07,-3.007992e+05,...,475.00,5.4686,2019,2019-01-03,5.3316,5.3412,6.1061,6.1171,6.7643,6.7996
2,2019-01-04,2.591328e+07,13775750.0,1.213753e+07,0.000000e+00,0.0,0.0,-1.098651e+08,-2.459390e+07,0.000000e+00,...,487.75,5.4186,2019,2019-01-04,5.4588,5.4686,6.2057,6.2169,6.8468,6.8825
3,2019-01-07,5.259345e+07,39041750.0,1.355170e+07,0.000000e+00,0.0,0.0,-5.165489e+07,-3.110594e+07,-3.364373e+06,...,496.25,5.3693,2019,2019-01-07,5.4088,5.4186,6.1687,6.1798,6.8370,6.8726
4,2019-01-08,3.650429e+07,15137500.0,2.136679e+07,0.000000e+00,0.0,0.0,-1.842178e+07,-1.246099e+07,-1.676525e+04,...,491.50,5.4362,2019,2019-01-08,5.3597,5.3693,6.1285,6.1395,6.8184,6.8539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,2023-01-26,1.408741e+08,90239000.0,4.977288e+07,8.622500e+05,0.0,0.0,-1.318398e+08,-4.748971e+07,-3.760017e+05,...,NaN,NaN,2023,2023-01-26,18.7850,18.8188,20.4370,20.4738,23.0914,23.2118
1021,2023-01-27,1.062562e+08,53296500.0,5.282375e+07,1.359018e+05,0.0,0.0,-2.777992e+08,-4.800505e+07,-1.360219e+08,...,NaN,NaN,2023,2023-01-27,18.7853,18.8191,20.4931,20.5301,23.2460,23.3671
1022,2023-01-30,5.225381e+08,350467750.0,1.679393e+08,4.131052e+06,0.0,0.0,-3.223933e+08,-1.854239e+08,-2.196258e+07,...,NaN,NaN,2023,2023-01-30,18.7852,18.8191,20.4401,20.4769,23.2050,23.3260
1023,2023-01-31,2.323393e+08,131395500.0,9.922482e+07,1.718965e+06,0.0,0.0,-1.187000e+08,-3.198080e+07,-1.251479e+06,...,NaN,NaN,2023,2023-01-31,18.7876,18.8215,20.4525,20.4894,23.2356,23.3568


In [1052]:
brent_cash_depo_otv_platts_usd.drop(['tarih_x', 'tarih_y'], axis= 1, inplace= True)

In [1053]:
volume.head()

,posting_date,product,material_description,category,sales_volumes_in_l_15
0,2019-01-01,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,935955.0000
1,2019-01-01,400003769,SH ULG 95 10ppmS E0 Udy Mkd FuelSave TR,Shell Fleet Solutions 100018,82.0275
2,2019-01-01,400003769,SH ULG 95 10ppmS E0 Udy Mkd FuelSave TR,Undercanopy 800001,2268.0725
3,2019-01-01,400003981,AGO 10ppmS B0 Udy Mkd FuelSave Diesel TR,Shell Fleet Solutions 100018,4625.2275
4,2019-01-01,400003981,AGO 10ppmS B0 Udy Mkd FuelSave Diesel TR,Undercanopy 800001,39751.6400


In [1054]:
volume['material_description'].unique()

array(['BF AGO 10ppmS B0 Udy Mkd Turkey',
       'SH ULG 95 10ppmS E0 Udy Mkd FuelSave TR',
       'AGO 10ppmS B0 Udy Mkd FuelSave Diesel TR', ...,
       'AC Paroil E_1*209L_A1MP', 'Alvania 0860_1*180kg_A1P5',
       'HlxUltProfAO-L 0W20_3*5L_EURO'], dtype=object)

In [1055]:
volume['category'].unique()

array(['Supply 310001', 'Shell Fleet Solutions 100018',
       'Undercanopy 800001', 'Lubricants 100034',
       'Commercial Fuels 800006', 'LSC 600010'], dtype=object)

In [1056]:
volume[(volume['material_description'] == 'BF AGO 10ppmS B0 Udy Mkd Turkey' )& (volume['category'] =='Supply 310001')]

,posting_date,product,material_description,category,sales_volumes_in_l_15
0,2019-01-01,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,9.359550e+05
24,2019-01-02,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,4.189095e+05
157,2019-01-03,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,2.828942e+05
294,2019-01-04,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,7.057548e+05
400,2019-01-05,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,4.236176e+06
...,...,...,...,...,...
148831,2023-01-27,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,2.009900e+04
148971,2023-01-28,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,5.386850e+04
148993,2023-01-30,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,7.152808e+05
149117,2023-01-31,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,9.914816e+04


In [1057]:
d = {'a' : volume['material_description'].unique(), 'b' :volume['category'].unique() }

In [1058]:
grid =  pd.MultiIndex.from_product(d.values(),names=d.keys()).to_frame().reset_index(drop=True)

In [1059]:
grid

,a,b
0,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001
1,BF AGO 10ppmS B0 Udy Mkd Turkey,Shell Fleet Solutions 100018
2,BF AGO 10ppmS B0 Udy Mkd Turkey,Undercanopy 800001
3,BF AGO 10ppmS B0 Udy Mkd Turkey,Lubricants 100034
4,BF AGO 10ppmS B0 Udy Mkd Turkey,Commercial Fuels 800006
...,...,...
6685,HlxUltProfAO-L 0W20_3*5L_EURO,Shell Fleet Solutions 100018
6686,HlxUltProfAO-L 0W20_3*5L_EURO,Undercanopy 800001
6687,HlxUltProfAO-L 0W20_3*5L_EURO,Lubricants 100034
6688,HlxUltProfAO-L 0W20_3*5L_EURO,Commercial Fuels 800006


In [1060]:
grid.merge(volume, left_on= ['a', 'b'], right_on= ['material_description', 'category'])

,a,b,posting_date,product,material_description,category,sales_volumes_in_l_15
0,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,2019-01-01,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,935955.00
1,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,2019-01-02,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,418909.50
2,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,2019-01-03,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,282894.25
3,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,2019-01-04,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,705754.75
4,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,2019-01-05,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,4236176.00
...,...,...,...,...,...,...,...
149369,Advance SX2Predl_12*1L_A926,LSC 600010,2023-01-11,550069610,Advance SX2Predl_12*1L_A926,LSC 600010,15.00
149370,AC Paroil E_1*209L_A1MP,Lubricants 100034,2022-12-08,550058411,AC Paroil E_1*209L_A1MP,Lubricants 100034,418.00
149371,Alvania 0860_1*180kg_A1P5,LSC 600010,2022-12-16,550069409,Alvania 0860_1*180kg_A1P5,LSC 600010,4050.00
149372,HlxUltProfAO-L 0W20_3*5L_EURO,Lubricants 100034,2023-01-09,550069004,HlxUltProfAO-L 0W20_3*5L_EURO,Lubricants 100034,7.50


In [1061]:
volume_fixed =  volume.pivot( columns= ['product', 'material_description', 'category'], values=  'sales_volumes_in_l_15', index= 'posting_date')

In [1062]:
volume_fixed = volume_fixed.droplevel(level=0, axis='columns')

In [1063]:
volume_fixed = volume_fixed.droplevel(level=0, axis='columns')

In [1064]:
volume_fixed 

category,Supply 310001,Shell Fleet Solutions 100018,Undercanopy 800001,Shell Fleet Solutions 100018,Undercanopy 800001,Undercanopy 800001,Supply 310001,Shell Fleet Solutions 100018,Undercanopy 800001,Shell Fleet Solutions 100018,...,LSC 600010,LSC 600010,Lubricants 100034,Lubricants 100034,LSC 600010,Lubricants 100034,LSC 600010,Lubricants 100034,LSC 600010,LSC 600010
posting_date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,9.359550e+05,82.0275,2268.0725,4625.2275,39751.6400,3928.75,18565.81525,1368.7725,5.360675e+04,11.4825,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,4.189095e+05,NaN,28243.7500,184.2500,265244.2500,248496.50,55268.79800,212.5000,8.376430e+05,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,2.828942e+05,13.2500,29391.2500,23.5000,318475.2500,297583.50,55116.98525,32.7500,8.841672e+05,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-04,7.057548e+05,NaN,45345.5000,NaN,423750.7500,304918.25,54648.15150,92.7650,1.129202e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-05,4.236176e+06,NaN,47572.7500,12016.0000,518194.2500,398006.75,45722.45025,1720.5000,1.358899e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-28,5.386850e+04,NaN,43633.0000,NaN,483387.7500,521717.50,26815.75000,NaN,1.738675e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-29,NaN,NaN,NaN,NaN,NaN,NaN,27003.50000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-30,7.152808e+05,NaN,49582.7500,NaN,513549.7500,485988.00,27683.50000,NaN,1.844408e+06,NaN,...,NaN,NaN,564.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1065]:
brent_cash_depo_otv_platts_usd_volume=  brent_cash_depo_otv_platts_usd.merge(volume_fixed, left_on= 'date', right_on= 'posting_date', how= 'left')

In [1066]:
brent_cash_depo_otv_platts_usd_volume

,date,total_inflows,customers_dds,customers_eft,t_&_s_collections,fx_sales,other_operations,total_outflows,tupras,other_oil,...,LSC 600010,LSC 600010,Lubricants 100034,Lubricants 100034,LSC 600010,Lubricants 100034,LSC 600010,Lubricants 100034,LSC 600010,LSC 600010
0,2019-01-02,1.747050e+07,7718750.0,9.655500e+06,0.000000e+00,0.0,96250.0,-4.631593e+07,-1.457022e+07,-1.441510e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-03,1.153048e+07,3431500.0,8.098978e+06,0.000000e+00,0.0,0.0,-1.535349e+07,-1.337181e+07,-3.007992e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-04,2.591328e+07,13775750.0,1.213753e+07,0.000000e+00,0.0,0.0,-1.098651e+08,-2.459390e+07,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-07,5.259345e+07,39041750.0,1.355170e+07,0.000000e+00,0.0,0.0,-5.165489e+07,-3.110594e+07,-3.364373e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-08,3.650429e+07,15137500.0,2.136679e+07,0.000000e+00,0.0,0.0,-1.842178e+07,-1.246099e+07,-1.676525e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,2023-01-26,1.408741e+08,90239000.0,4.977288e+07,8.622500e+05,0.0,0.0,-1.318398e+08,-4.748971e+07,-3.760017e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1021,2023-01-27,1.062562e+08,53296500.0,5.282375e+07,1.359018e+05,0.0,0.0,-2.777992e+08,-4.800505e+07,-1.360219e+08,...,1080.0,4050.0,NaN,NaN,NaN,NaN,NaN,NaN,315.0,NaN
1022,2023-01-30,5.225381e+08,350467750.0,1.679393e+08,4.131052e+06,0.0,0.0,-3.223933e+08,-1.854239e+08,-2.196258e+07,...,NaN,NaN,564.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1023,2023-01-31,2.323393e+08,131395500.0,9.922482e+07,1.718965e+06,0.0,0.0,-1.187000e+08,-3.198080e+07,-1.251479e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1067]:
brent_cash_depo_otv_platts_usd_volume.isnull().sum().sort_values(ascending= False)

Lubricants 100034          1025
Lubricants 100034          1025
LSC 600010                 1025
LSC 600010                 1024
Lubricants 100034          1024
                           ... 
gbp_alis                      0
gbp_satis                     0
Commercial Fuels 800006       0
Commercial Fuels 800006       0
date                          0
Length: 1654, dtype: int64

In [1068]:
brent_cash_depo_otv_platts_usd_volume.shape

(1025, 1654)

In [1069]:
limit_par = len(brent_cash_depo_otv_platts_usd_volume) * 0.7
brent_cash_depo_otv_platts_usd_volume.dropna(thresh= limit_par, axis=1, inplace= True)

In [1070]:
brent_cash_depo_otv_platts_usd

,date,total_inflows,customers_dds,customers_eft,t_&_s_collections,fx_sales,other_operations,total_outflows,tupras,other_oil,...,ab_piyasa_fiyat_dusuk,dolar_kuru_sats,yl,tarih,usd_alis,usd_satis,eur_alis,eur_satis,gbp_alis,gbp_satis
0,2019-01-02,1.747050e+07,7718750.0,9.655500e+06,0.000000e+00,0.0,96250.0,-4.631593e+07,-1.457022e+07,-1.441510e+05,...,485.25,5.3412,2019,2019-01-02,5.2810,5.2905,6.0422,6.0530,6.7135,6.7485
1,2019-01-03,1.153048e+07,3431500.0,8.098978e+06,0.000000e+00,0.0,0.0,-1.535349e+07,-1.337181e+07,-3.007992e+05,...,475.00,5.4686,2019,2019-01-03,5.3316,5.3412,6.1061,6.1171,6.7643,6.7996
2,2019-01-04,2.591328e+07,13775750.0,1.213753e+07,0.000000e+00,0.0,0.0,-1.098651e+08,-2.459390e+07,0.000000e+00,...,487.75,5.4186,2019,2019-01-04,5.4588,5.4686,6.2057,6.2169,6.8468,6.8825
3,2019-01-07,5.259345e+07,39041750.0,1.355170e+07,0.000000e+00,0.0,0.0,-5.165489e+07,-3.110594e+07,-3.364373e+06,...,496.25,5.3693,2019,2019-01-07,5.4088,5.4186,6.1687,6.1798,6.8370,6.8726
4,2019-01-08,3.650429e+07,15137500.0,2.136679e+07,0.000000e+00,0.0,0.0,-1.842178e+07,-1.246099e+07,-1.676525e+04,...,491.50,5.4362,2019,2019-01-08,5.3597,5.3693,6.1285,6.1395,6.8184,6.8539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,2023-01-26,1.408741e+08,90239000.0,4.977288e+07,8.622500e+05,0.0,0.0,-1.318398e+08,-4.748971e+07,-3.760017e+05,...,NaN,NaN,2023,2023-01-26,18.7850,18.8188,20.4370,20.4738,23.0914,23.2118
1021,2023-01-27,1.062562e+08,53296500.0,5.282375e+07,1.359018e+05,0.0,0.0,-2.777992e+08,-4.800505e+07,-1.360219e+08,...,NaN,NaN,2023,2023-01-27,18.7853,18.8191,20.4931,20.5301,23.2460,23.3671
1022,2023-01-30,5.225381e+08,350467750.0,1.679393e+08,4.131052e+06,0.0,0.0,-3.223933e+08,-1.854239e+08,-2.196258e+07,...,NaN,NaN,2023,2023-01-30,18.7852,18.8191,20.4401,20.4769,23.2050,23.3260
1023,2023-01-31,2.323393e+08,131395500.0,9.922482e+07,1.718965e+06,0.0,0.0,-1.187000e+08,-3.198080e+07,-1.251479e+06,...,NaN,NaN,2023,2023-01-31,18.7876,18.8215,20.4525,20.4894,23.2356,23.3568


In [1071]:
brent_cash_depo_otv_platts_usd_volume.head()

,date,total_inflows,customers_dds,customers_eft,t_&_s_collections,fx_sales,other_operations,total_outflows,tupras,other_oil,...,Commercial Fuels 800006,Commercial Fuels 800006,Commercial Fuels 800006,Undercanopy 800001,Commercial Fuels 800006,Commercial Fuels 800006,Undercanopy 800001,LSC 600010,Lubricants 100034,Lubricants 100034
0,2019-01-02,1.747050e+07,7718750.0,9.655500e+06,0.0,0.0,96250.0,-4.631593e+07,-1.457022e+07,-144151.00,...,420089.00,10315.75,80332.25,120440.25,6034.25,8936.5,560537.00,365.75,313.50,NaN
1,2019-01-03,1.153048e+07,3431500.0,8.098978e+06,0.0,0.0,0.0,-1.535349e+07,-1.337181e+07,-300799.25,...,378238.25,20507.00,75298.75,121302.50,2230.25,7594.5,586303.50,NaN,1254.00,NaN
2,2019-01-04,2.591328e+07,13775750.0,1.213753e+07,0.0,0.0,0.0,-1.098651e+08,-2.459390e+07,0.00,...,415215.50,13539.75,39619.00,161824.00,21196.25,4947.5,689547.25,NaN,1045.00,NaN
3,2019-01-07,5.259345e+07,39041750.0,1.355170e+07,0.0,0.0,0.0,-5.165489e+07,-3.110594e+07,-3364373.00,...,364928.50,16551.75,76796.25,172257.50,752.00,13381.0,667081.50,NaN,2246.75,NaN
4,2019-01-08,3.650429e+07,15137500.0,2.136679e+07,0.0,0.0,0.0,-1.842178e+07,-1.246099e+07,-16765.25,...,491370.25,7917.75,97127.75,232997.75,5780.50,4348.5,759424.75,6479.00,104.50,NaN


In [1072]:
sample_submission.head()

,Date,Net Cashflow from Operations
0,2023-02-02,0
1,2023-02-03,0
2,2023-02-06,0
3,2023-02-07,0
4,2023-02-08,0


In [1073]:
sample_submission.tail()

,Date,Net Cashflow from Operations
65,2023-05-08,0
66,2023-05-09,0
67,2023-05-10,0
68,2023-05-11,0
69,2023-05-12,0


In [1074]:
sample_submission['Date'].tail(1)

69    2023-05-12
Name: Date, dtype: object

In [1075]:
pd.to_datetime(sample_submission['Date'].tail(1))  - brent_cash_depo_otv_platts_usd_volume['date'].tail(1)

69     NaT
1024   NaT
dtype: timedelta64[ns]

In [1076]:
brent_cash_depo_otv_platts_usd_volume['date']

0      2019-01-02
1      2019-01-03
2      2019-01-04
3      2019-01-07
4      2019-01-08
          ...    
1020   2023-01-26
1021   2023-01-27
1022   2023-01-30
1023   2023-01-31
1024   2023-02-01
Name: date, Length: 1025, dtype: datetime64[ns]

In [1077]:
brent_cash_depo_otv_platts_usd_volume.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1025 entries, 0 to 1024
Data columns (total 70 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            1025 non-null   datetime64[ns]
 1   total_inflows                   1025 non-null   float64       
 2   customers_dds                   1025 non-null   float64       
 3   customers_eft                   1025 non-null   float64       
 4   t_&_s_collections               1025 non-null   float64       
 5   fx_sales                        1025 non-null   float64       
 6   other_operations                1025 non-null   float64       
 7   total_outflows                  1025 non-null   float64       
 8   tupras                          1025 non-null   float64       
 9   other_oil                       1025 non-null   float64       
 10  gas                             1025 non-null   float64       
 11  impo

In [1078]:
brent_cash_depo_otv_platts_usd_volume['date'] + pd.Timedelta(days= 105)

0      2019-04-17
1      2019-04-18
2      2019-04-19
3      2019-04-22
4      2019-04-23
          ...    
1020   2023-05-11
1021   2023-05-12
1022   2023-05-15
1023   2023-05-16
1024   2023-05-17
Name: date, Length: 1025, dtype: datetime64[ns]

In [1079]:
sample_submission

,Date,Net Cashflow from Operations
0,2023-02-02,0
1,2023-02-03,0
2,2023-02-06,0
3,2023-02-07,0
4,2023-02-08,0
...,...,...
65,2023-05-08,0
66,2023-05-09,0
67,2023-05-10,0
68,2023-05-11,0


In [1080]:
brent_cash_depo_otv_platts_usd_volume.shift(58)

,date,total_inflows,customers_dds,customers_eft,t_&_s_collections,fx_sales,other_operations,total_outflows,tupras,other_oil,...,Commercial Fuels 800006,Commercial Fuels 800006,Commercial Fuels 800006,Undercanopy 800001,Commercial Fuels 800006,Commercial Fuels 800006,Undercanopy 800001,LSC 600010,Lubricants 100034,Lubricants 100034
0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,2022-11-07,4.382662e+08,124369250.0,2.003705e+08,6.027646e+07,0.0,53250000.0,-3.661203e+08,-1.860444e+08,-5.941592e+07,...,460187.75,36395.50,46318.50,519492.00,52906.50,2997.25,1322290.75,156.75,NaN,5753.91900
1021,2022-11-08,1.884265e+08,90559750.0,7.237078e+07,1.349596e+07,0.0,12000000.0,-1.436337e+08,-5.510235e+07,-6.147685e+07,...,296167.75,35344.75,10789.25,396670.50,23601.75,11224.00,1023711.25,261.25,418.00,10742.72200
1022,2022-11-09,1.479083e+08,57181500.0,6.360692e+07,1.386991e+07,0.0,13250000.0,-9.762621e+07,-6.291959e+07,-2.070314e+07,...,274580.50,16222.50,43884.50,329403.25,30955.50,NaN,840210.50,261.25,836.00,47654.88875
1023,2022-11-10,2.974818e+08,159792250.0,1.003952e+08,3.729432e+07,0.0,0.0,-1.789509e+08,-9.556009e+06,-1.195034e+08,...,306528.25,32868.75,10848.50,390801.50,33395.00,1674.75,947932.25,261.25,783.75,28072.90375


In [1081]:
brent_cash_depo_otv_platts_usd_volume

,date,total_inflows,customers_dds,customers_eft,t_&_s_collections,fx_sales,other_operations,total_outflows,tupras,other_oil,...,Commercial Fuels 800006,Commercial Fuels 800006,Commercial Fuels 800006,Undercanopy 800001,Commercial Fuels 800006,Commercial Fuels 800006,Undercanopy 800001,LSC 600010,Lubricants 100034,Lubricants 100034
0,2019-01-02,1.747050e+07,7718750.0,9.655500e+06,0.000000e+00,0.0,96250.0,-4.631593e+07,-1.457022e+07,-1.441510e+05,...,420089.00,10315.75,80332.25,120440.25,6034.25,8936.5,560537.00,365.75,313.50,NaN
1,2019-01-03,1.153048e+07,3431500.0,8.098978e+06,0.000000e+00,0.0,0.0,-1.535349e+07,-1.337181e+07,-3.007992e+05,...,378238.25,20507.00,75298.75,121302.50,2230.25,7594.5,586303.50,NaN,1254.00,NaN
2,2019-01-04,2.591328e+07,13775750.0,1.213753e+07,0.000000e+00,0.0,0.0,-1.098651e+08,-2.459390e+07,0.000000e+00,...,415215.50,13539.75,39619.00,161824.00,21196.25,4947.5,689547.25,NaN,1045.00,NaN
3,2019-01-07,5.259345e+07,39041750.0,1.355170e+07,0.000000e+00,0.0,0.0,-5.165489e+07,-3.110594e+07,-3.364373e+06,...,364928.50,16551.75,76796.25,172257.50,752.00,13381.0,667081.50,NaN,2246.75,NaN
4,2019-01-08,3.650429e+07,15137500.0,2.136679e+07,0.000000e+00,0.0,0.0,-1.842178e+07,-1.246099e+07,-1.676525e+04,...,491370.25,7917.75,97127.75,232997.75,5780.50,4348.5,759424.75,6479.00,104.50,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,2023-01-26,1.408741e+08,90239000.0,4.977288e+07,8.622500e+05,0.0,0.0,-1.318398e+08,-4.748971e+07,-3.760017e+05,...,130759.00,25125.00,NaN,28334.75,NaN,NaN,51036.50,8778.00,992.75,17423.48850
1021,2023-01-27,1.062562e+08,53296500.0,5.282375e+07,1.359018e+05,0.0,0.0,-2.777992e+08,-4.800505e+07,-1.360219e+08,...,263358.50,29175.25,NaN,40980.75,NaN,NaN,118578.75,4284.50,731.50,19849.46500
1022,2023-01-30,5.225381e+08,350467750.0,1.679393e+08,4.131052e+06,0.0,0.0,-3.223933e+08,-1.854239e+08,-2.196258e+07,...,281245.75,24063.00,NaN,95048.00,NaN,NaN,243090.25,104.50,418.00,13989.79850
1023,2023-01-31,2.323393e+08,131395500.0,9.922482e+07,1.718965e+06,0.0,0.0,-1.187000e+08,-3.198080e+07,-1.251479e+06,...,294369.25,36002.75,NaN,92303.25,NaN,NaN,241450.00,104.50,NaN,NaN


In [1082]:
brent_cash_depo_otv_platts_usd_volume['date_'] = brent_cash_depo_otv_platts_usd_volume['date'] + pd.Timedelta(days=105)

In [1083]:
brent_cash_depo_otv_platts_usd_volume[['date','date_']]

,date,date_
0,2019-01-02,2019-04-17
1,2019-01-03,2019-04-18
2,2019-01-04,2019-04-19
3,2019-01-07,2019-04-22
4,2019-01-08,2019-04-23
...,...,...
1020,2023-01-26,2023-05-11
1021,2023-01-27,2023-05-12
1022,2023-01-30,2023-05-15
1023,2023-01-31,2023-05-16


In [1084]:
brent_cash_depo_otv_platts_usd_volume.set_index('date_', inplace= True)

In [1085]:
data = brent_cash_depo_otv_platts_usd_volume._get_numeric_data()

In [1086]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1025 entries, 2019-04-17 to 2023-05-17
Data columns (total 59 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   total_inflows                   1025 non-null   float64
 1   customers_dds                   1025 non-null   float64
 2   customers_eft                   1025 non-null   float64
 3   t_&_s_collections               1025 non-null   float64
 4   fx_sales                        1025 non-null   float64
 5   other_operations                1025 non-null   float64
 6   total_outflows                  1025 non-null   float64
 7   tupras                          1025 non-null   float64
 8   other_oil                       1025 non-null   float64
 9   gas                             1025 non-null   float64
 10  import_payments_fx_purchases    1025 non-null   float64
 11  tax                             1025 non-null   float64
 12  operational_and_

In [1087]:
# data['day_of_the_week'] = data.index.day_of_week
# data['month'] = data.index.month

In [1088]:
data_train =  data['2019-04-02':'2023-02-01']

In [1089]:
data_test = data['2023-02-02': '2023-05-12']

In [1090]:
data_test.drop('net_cashflow_from_operations', axis=1, inplace= True)

/var/folders/t1/db5m68vn10v3148_nq_7r8b80000gn/T/ipykernel_1061/2263494553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test.drop('net_cashflow_from_operations', axis=1, inplace= True)


In [1091]:
X = data_train.drop('net_cashflow_from_operations', axis=1)

In [1092]:
y = data_train['net_cashflow_from_operations']

In [1093]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42, shuffle=False)

In [1094]:
pipe = Pipeline([('impute', KNNImputer()), ('pca', PCA(n_components=12)),
                  ('lgbm', LGBMRegressor())])


In [1095]:
np.random.seed(42)
pipe.fit(X,y)

Pipeline(steps=[('impute', KNNImputer()), ('pca', PCA(n_components=15)),
                ('lgbm', LGBMRegressor())])

In [1096]:
np.random.seed(42)
prediction = pipe.predict(X_test)

In [1097]:
mean_absolute_error(y_test, prediction)

5155953.526291144

In [1098]:
sub = pipe.predict(data_test)

In [1099]:
submission = pd.DataFrame({'Date' :  data_test.index, 'Net Cashflow from Operations' : sub})

In [1100]:
submission['Date'] = submission['Date'].astype('str')

In [1101]:
submission['Net Cashflow from Operations'] = submission['Net Cashflow from Operations'].astype('int64')

In [1102]:
submission.drop(labels= [56,62], axis= 0, inplace= True)

In [1103]:
pd.DataFrame.to_csv(submission, 'submission.csv', index= False)

In [1104]:
submission

,Date,Net Cashflow from Operations
0,2023-02-02,89818905
1,2023-02-03,-148665575
2,2023-02-06,-63888916
3,2023-02-07,73585391
4,2023-02-08,15670394
...,...,...
67,2023-05-08,-116665324
68,2023-05-09,37448482
69,2023-05-10,24064044
70,2023-05-11,6874192


In [1105]:
sample_submission

,Date,Net Cashflow from Operations
0,2023-02-02,0
1,2023-02-03,0
2,2023-02-06,0
3,2023-02-07,0
4,2023-02-08,0
...,...,...
65,2023-05-08,0
66,2023-05-09,0
67,2023-05-10,0
68,2023-05-11,0
